In [1]:
import sys
sys.path.insert(0, '/Users/alam/code/vfftn/src')
from models2 import heisenberg_ham, kronecker_pad 
from mpo import pauli
import numpy as np 
from scipy.special import comb
from functools import reduce
from scipy.optimize import minimize
from itertools import permutations
from tqdm.notebook import tqdm 

# xxx model
xxx_ham = lambda n : heisenberg_ham(n, jx=1/4,jy=1/4,jz=1/4,h=0)  

In [2]:
def total_sz(n): 
    total_z = np.zeros((2**n,2**n), dtype=complex)
    for i in range(n): 
        total_z += kronecker_pad(pauli[3], n, i)
    return total_z

def total_spin(n): 
    total_x = np.zeros((2**n,2**n), dtype=complex)
    total_y = np.zeros((2**n,2**n), dtype=complex)
    total_z = np.zeros((2**n,2**n), dtype=complex)
    for i in range(n):
        total_x += kronecker_pad(pauli[1], n, i) 
        total_y += kronecker_pad(pauli[2], n, i)
        total_z += kronecker_pad(pauli[3], n, i)

    return [total_x, total_y, total_z]

def total_spin_squared(n):
    total_x, total_y, total_z = total_spin(n)
    return total_x @ total_x + total_y @ total_y + total_z @ total_z

def su2_multiplicity(n,j):
    """ returns the multiplicity of the j-th irrep of n tensor products of su(2) """
    return int(comb(n,int(n/2-j)) - comb(n, int(n/2-j)-1)) 

def su2_dimension(j): 
    return int(2*j + 1)  

def projector(n,j,m): 
    """ returns the projector onto the subspace with total spin j and total z-component m """
    U = np.loadtxt(f'N={n}.txt', delimiter=',')
    
    j_cur = 0 if n % 2 == 0 else 1/2
    sector_start = 0 
    while j_cur < j: 
        sector_start += su2_multiplicity(n,j_cur) * su2_dimension(j_cur)
        j_cur += 1

    m_offset = int((-m * 2 + 2*j)/2) if (m*2)%2 == 1 else (-m+j)
    indices = []
    for i in range(su2_multiplicity(n,j)):
        indices.append(sector_start + i*su2_dimension(j) + m_offset)

    P = np.zeros((2**n,2**n), dtype=complex)
    for index in indices: 
        P += np.outer(U[:,index], U[:,index].conj())
    return P, U[:,indices] 

def nn_ansatz(params, n): 
    ansatz = np.zeros((2**n,2**n), dtype=complex)

    for i in range(n):
        ansatz += params[i] * kronecker_pad(pauli[3], n, i) 

    ZZ = np.kron(pauli[3], pauli[3])
    for i in range(n-1):
        ansatz += params[n+i] * kronecker_pad(ZZ, n, i)
        
    ansatz += params[-1] * np.eye(2**n)
    return ansatz 

def nnn_ansatz(params, n):
    """ diagonal unitary built from nearest-neighbor and next-nearest-neighbor Z interactions """
    ansatz = np.zeros((2**n,2**n), dtype=complex)

    for i in range(n):
        ansatz += params[i] * kronecker_pad(pauli[3], n, i) 

    ZZ = np.kron(pauli[3], pauli[3])
    for i in range(n-1):
        ansatz += params[n+i] * kronecker_pad(ZZ, n, i)
        
    ZZZ = np.kron(ZZ, pauli[3])
    for i in range(n-2):
        ansatz += params[2*n-1+i] * kronecker_pad(ZZZ, n, i)

    ansatz += params[-1] * np.eye(2**n)
    return ansatz

def two_local_ansatz(params, n): 
    ansatz = np.zeros((2**n,2**n), dtype=complex)

    for i in range(n):
        ansatz += params[i] * kronecker_pad(pauli[3], n, i) 

    # generates pairs of sites from 0 to n-1
    sites = [(i,j) for i in range(n) for j in range(i+1,n)]
    for param_count, (i,j) in enumerate(sites): 
        kron_list = [pauli[3] if k == i or k == j else np.eye(2) for k in range(n)]
        ansatz += params[n + param_count] * reduce(np.kron, kron_list)

    ansatz += params[-1] * np.eye(2**n)
    return ansatz 

def unitary_equivalence(D, H):
    """
    Finds a unitary matrix U such that U D U^† = H. D and H are normal matrices with the same eigenvalues. 
    """
    # Perform eigenvalue decomposition for D and H
    eigvals_D, V_D = np.linalg.eigh(D)  # eigenvalues and eigenvectors of D
    eigvals_H, V_H = np.linalg.eigh(H)  # eigenvalues and eigenvectors of H
    
    # Ensure eigenvalues are sorted consistently (since they might not be in order)
    idx_D = np.argsort(eigvals_D)
    idx_H = np.argsort(eigvals_H)
    
    V_D = V_D[:, idx_D]
    V_H = V_H[:, idx_H]
    
    # Construct U
    U = V_H @ V_D.conj().T
    
    return U

def nn_cost(params, n, target_evs):
    ansatz = basis.conj().T @ nn_ansatz(params, n) @ basis
    ansatz_evals = np.linalg.eigvalsh(ansatz)
    return np.linalg.norm(ansatz_evals - target_evs)

def nnn_cost(params, n, target_evs):
    ansatz = basis.conj().T @ nnn_ansatz(params, n) @ basis
    ansatz_evals = np.linalg.eigvalsh(ansatz)
    return np.linalg.norm(ansatz_evals - target_evs)

def two_local_cost(params, n, target_evs): 
    ansatz = basis.conj().T @ two_local_ansatz(params, n) @ basis
    ansatz_evals = np.linalg.eigvalsh(ansatz)
    return np.linalg.norm(ansatz_evals - target_evs)

def create_callback(n, target_evs, cost_func):
    def callback(xk):
        current_cost = cost_func(xk, n, target_evs)
        print(f"Current cost: {current_cost}")
    return callback


# NNN

In [50]:
n = 8
j = 1
m = 1

Sz = total_sz(n)
S_sq = total_spin_squared(n)
P, basis = projector(n,j,m)
print(basis.shape[1]) 

28


In [51]:
eff_ham = basis.conj().T @ xxx_ham(n) @ basis
target_evals = np.linalg.eigvalsh(eff_ham)

In [56]:
# nnn ansatz

initial_params = np.random.rand(3*n - 2)
print(len(initial_params))
result = minimize(nnn_cost, initial_params, args=(n, target_evals), method='L-BFGS-B', 
                  callback=create_callback(n, target_evals))

22
Current cost: 5.280209580947351
Current cost: 2.3022493835472204
Current cost: 1.4633960807397486
Current cost: 0.7014194505830292
Current cost: 0.4701510651113194
Current cost: 0.3902861193003509
Current cost: 0.36012747214638674
Current cost: 0.32473777128044207
Current cost: 0.2993980918217359
Current cost: 0.26691959568269386
Current cost: 0.2382144926237786
Current cost: 0.21989804718435302
Current cost: 0.20712047000654507
Current cost: 0.2011295119937502
Current cost: 0.19251577578080115
Current cost: 0.18454667772635222
Current cost: 0.176685654242638
Current cost: 0.171132442261842
Current cost: 0.16408821444044136
Current cost: 0.15948104357540618
Current cost: 0.15033162739500208
Current cost: 0.14209458328712393
Current cost: 0.1363642817953492
Current cost: 0.13384703724700864
Current cost: 0.1323116352865512
Current cost: 0.13164581529445432
Current cost: 0.13119658412761911
Current cost: 0.1300317677609216
Current cost: 0.12740793177683776
Current cost: 0.126379693053

In [ ]:
# nnn ansatz, with (8,1,1) 
final_cost = [0.065798, 0.058938, 0.068421, 0.100569, 0.0863811]

# Two local

In [14]:
n = 8
j = 1
m = 1

Sz = total_sz(n)
S_sq = total_spin_squared(n)
P, basis = projector(n,j,m)
print(basis.shape[1]) 

28


In [15]:
eff_ham = basis.conj().T @ xxx_ham(n) @ basis
target_evals = np.linalg.eigvalsh(eff_ham)

In [19]:
initial_params = np.random.rand(n+1 + (n-1)*(n-2))
print(len(initial_params))

result = minimize(two_local_cost, initial_params, args=(n, target_evals), method='L-BFGS-B', 
                  callback=create_callback(n, target_evals, two_local_cost)) 

51
Current cost: 1.6690593213064253
Current cost: 0.7022955350246942
Current cost: 0.5636556180768174
Current cost: 0.46707217065693596
Current cost: 0.3436141197115947
Current cost: 0.3140151545829869
Current cost: 0.2689266950119889
Current cost: 0.23855758225846463
Current cost: 0.22037570209495566
Current cost: 0.17525917342370978
Current cost: 0.16087355391685598
Current cost: 0.14059059446709282
Current cost: 0.12510247554455386
Current cost: 0.10967621810169569
Current cost: 0.09799565486506386
Current cost: 0.09025364385699872
Current cost: 0.0825923785299938
Current cost: 0.07577635125498962
Current cost: 0.07220713814826173
Current cost: 0.0693341321165476
Current cost: 0.06835479503557916
Current cost: 0.06661999212519913
Current cost: 0.0635751339970852
Current cost: 0.06242082834443797
Current cost: 0.059485503947017
Current cost: 0.056916543976460815
Current cost: 0.05529862244935012
Current cost: 0.052130029678515426
Current cost: 0.049072500864781966
Current cost: 0.046

In [ ]:
# two_local ansatz, with (8,1,1)

final_costs = [0.026238, 0.018974, 0.02909, 0.025780, 0.029]